## Проанализируем группы клиентов и постараемся ответим на вопрос, какой тип рекламы успешнее. Все выводы об эффективности подтвердим статистическими тестами.

### PS: я очень долго не мог вникнуть в эту тему, 2 раза пересматривал лекции, надеюсь я все правильно понял и сделал....

### Загрузка и подготовка данных

In [1]:
# Подключаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [3]:
# Считываем csv файл с данными

df = pd.read_csv(r"D:\marketing_AB.csv")
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [5]:
# Создадим новый столбец, в котором укажем какой именно день наибольшего количества рекламы - будни или выходные
df['days'] = np.where((df['most ads day'] == 'Saturday')|(df['most ads day'] == 'Sunday'), 'weekend', 'work days')
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour,days
0,0,1069124,ad,False,130,Monday,20,work days
1,1,1119715,ad,False,93,Tuesday,22,work days
2,2,1144181,ad,False,21,Tuesday,18,work days
3,3,1435133,ad,False,355,Tuesday,10,work days
4,4,1015700,ad,False,276,Friday,14,work days


In [7]:
df['total ads'].describe()

count    588101.000000
mean         24.820876
std          43.715181
min           1.000000
25%           4.000000
50%          13.000000
75%          27.000000
max        2065.000000
Name: total ads, dtype: float64

In [9]:
df.shape

(588101, 8)

In [11]:
# Создадим датасет по группе А - те, кто видел базовую рекламу ad
group_a = df[df['test group'] == "ad"]
group_a.shape

(564577, 8)

In [13]:
# Создадим датасет по группе B - те, кто видел социальную рекламу psa
group_b = df[df['test group'] == "psa"]
group_b.shape

(23524, 8)

In [15]:
# Количество людей, которые купили продукт
df[df['converted'] == True].shape

(14843, 8)

In [17]:
# Количество людей из группы А, которые купили продукт
group_a[group_a['converted'] == True].shape

(14423, 8)

In [19]:
# Количество людей из группы B, которые купили продукт
group_b[group_b['converted'] == True].shape

(420, 8)

### Расчет конверсии

In [21]:
print('Конверсия общая:', df[df['converted'] == True].shape[0] / df.shape[0])                  # 14843/588101
print('Конверсия группы А:', group_a[group_a['converted'] == True].shape[0] / group_a.shape[0])   # 14423/564577
print('Конверсия группы Б:', group_b[group_b['converted'] == True].shape[0] / group_b.shape[0])   # 420/23524

Конверсия общая: 0.02523886203220195
Конверсия группы А: 0.025546559636683747
Конверсия группы Б: 0.01785410644448223


### Тест на значимость разницы между группами

In [23]:
stats.ttest_ind(group_a['converted'], group_b['converted'])

TtestResult(statistic=7.37040597428566, pvalue=1.7033052627831264e-13, df=588099.0)

##### Нулевая гипотеза: равенство средней конверсии в обеих группах. Сравниваем p-значение (13 нулей после запятой) и альфа (5%), p-значение < 5%, поэтому отвергаем эту гипотезу. Конверсия в группе А, которой показывали базовую рекламу, и конверсия в группе B, которой показывали социальную рекламу, отличаются. Для увеличения показателя покупки продукта рекомендуется показывать базовую рекламу ad.


### Выберем рекламу ad. Посчитаем в каких пределах может находиться конверсия с вероятностью 95%.

In [25]:
mean = group_a['converted'].mean()   # Среднее значение
var = group_a['converted'].var()     # Дисперсия
n = group_a['converted'].count()     # Количество значений 

ci = stats.norm.interval(confidence = 0.95, loc = mean, scale = np.sqrt(var/n))
print(f'95% доверительный интервал для конверсии рекламы ad: от {ci[0].round(3)} до {ci[1].round(3)}')

95% доверительный интервал для конверсии рекламы ad: от 0.025 до 0.026


### Выберем рекламу psa. Посчитаем в каких пределах может находиться конверсия с вероятностью 95%.

In [27]:
mean = group_b['converted'].mean()   # Среднее значение
var = group_b['converted'].var()     # Дисперсия
n = group_b['converted'].count()     # Количество значений 

ci = stats.norm.interval(confidence = 0.95, loc = mean, scale = np.sqrt(var/n))
print(f'95% доверительный интервал для конверсии рекламы psa: от {ci[0].round(3)} до {ci[1].round(3)}')

95% доверительный интервал для конверсии рекламы psa: от 0.016 до 0.02


### Посмотрим на гетерогенность эффектов в группах А и В

##### Для этого я специально создал вначале новый столбец, в котором указал день наибольшего количества рекламы - будни или выходные

In [29]:
# Сгруппируем по будням и выходным и расчитаем конверсию для группы А
group_a.groupby('days')['converted'].mean()

days
weekend      0.022999
work days    0.026564
Name: converted, dtype: float64

In [31]:
# Сгруппируем по будням и выходным и расчитаем конверсию для группы B
group_b.groupby('days')['converted'].mean()

days
weekend      0.017407
work days    0.018004
Name: converted, dtype: float64

### Проведем тест на значимость эффекта отдельно в выходные дни

In [33]:
stats.ttest_ind(group_a[group_a['days'] == 'weekend']['converted'], group_b[group_b['days'] == 'weekend']['converted'])

TtestResult(statistic=2.830245399653293, pvalue=0.004651786243671406, df=167049.0)

##### p-значение маленькое, средние конверсии отличаются, значит для увеличения показателя покупки продукта в выходные дни рекомендуется показывать базовую рекламу ad, т.к. конверсия группы А в выходные дни выше

### Проведем тест на значимость эффекта отдельно в будние дни

In [35]:
stats.ttest_ind(group_a[group_a['days'] == 'work days']['converted'], group_b[group_b['days'] == 'work days']['converted'])

TtestResult(statistic=6.960070093276779, pvalue=3.4059707794507603e-12, df=421048.0)

##### p-значение маленькое, средние конверсии отличаются, значит для увеличения показателя покупки продукта в будние дни рекомендуется показывать базовую рекламу ad, так как конверсия группы А в будние дни выше

### Посмотрим на гетерогенность эффекта в группах А и В и проведем тест на значимость эффекта по количеству проссмотров рекламы, а именно менее и более 200 раз

#### Более 200 раз:

In [37]:
# Посчитаем количество человек из групп А и B, которые проссмотрели рекламу более 200 раз
ga = group_a[group_a['total ads'] > 200].shape[0]
gb = group_b[group_b['total ads'] > 200].shape[0]
print(f'Количество человек, проссмотревших рекламу более 200 раз:')
print(f'Группа А - {ga}')
print(f'Группа B - {gb}')

Количество человек, проссмотревших рекламу более 200 раз:
Группа А - 5694
Группа B - 258


In [39]:
# Расчитаем конверсию для групп А и B, по людям, проссмотревшим рекламу более 200 раз
gaa = group_a[group_a['total ads'] > 200]['converted'].mean()
gbb = group_b[group_b['total ads'] > 200]['converted'].mean()
print(f'Конверсия:')
print(f'Группа А - {gaa}')
print(f'Группа B - {gbb}')

Конверсия:
Группа А - 0.15577801194239552
Группа B - 0.15503875968992248


In [41]:
# Посчитаем статистику между группами людей, которые проссмотрели рекламу более 200 раз
stats.ttest_ind(group_a[group_a['total ads'] > 200]['converted'], group_b[group_b['total ads'] > 200]['converted'])

TtestResult(statistic=0.03202298862130002, pvalue=0.9744547921122856, df=5950.0)

##### p-значение большое, средние конверсии не отличаются

#### Менее 200 раз:

In [44]:
print(f'Количество человек, проссмотревших рекламу более 200 раз:')
print(f'Группа А - {group_a[group_a['total ads'] < 200].shape[0]}')
print(f'Группа B - {group_b[group_b['total ads'] < 200].shape[0]}')
print(f'\nКонверсия:')
print(f'Группа А - {group_a[group_a['total ads'] < 200]['converted'].mean()}')
print(f'Группа B - {group_b[group_b['total ads'] < 200]['converted'].mean()}')

Количество человек, проссмотревших рекламу более 200 раз:
Группа А - 558837
Группа B - 23263

Конверсия:
Группа А - 0.02420920590440504
Группа B - 0.016291965782573185


In [46]:
stats.ttest_ind(group_a[group_a['total ads'] < 200]['converted'], group_b[group_b['total ads'] < 200]['converted'])

TtestResult(statistic=7.748003386239436, pvalue=9.349802016642577e-15, df=582098.0)

##### p-значение маленькое, средние конверсии отличаются

#### Выводы:
#### Если людям давать смотреть рекламу менее 200 раз, то нужно давать смотреть базовую рекламу ad для увеличесния количества продаж.
#### Если людям давать смотреть рекламу более 200 раз, то без разницы какую рекламу им давать смотреть, показатель количества покупок будет одинаковый.